# Import Dataset and Libraries

In [1]:
# %pip install imbalanced-learn==0.10.1
# %pip install catboost==1.1.1
# %pip install pycountry==22.1.10
# %pip install scikit-learn==1.2.2

In [2]:
import numpy as np
import pandas as pd

RANDOM_STATE = 42
TRAIN_FILE = "./train.csv"
TEST_FILE = "./submission.csv"

In [3]:
df_train = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE)
df_submission = pd.read_csv(TEST_FILE)

X, y = df_train.drop("is_converted", axis=1), df_train["is_converted"]
X_test = df_test.drop(["is_converted", "id"], axis=1)

# Data Preprocessing

## 1. Data Imputation

In [4]:
values = {
    **{column: 0 for column in X.columns},
    "inquiry_type": "Quotation or Purchase Consultation",
    "customer_country": "Not Found",
    "historical_existing_cnt": X["historical_existing_cnt"].mode()[0],
}

X.fillna(value=values, inplace=True)
X_test.fillna(value=values, inplace=True)

## 2. Data Cleaning 

### Feature: Customer Country

In [5]:
import pycountry


def preprocess_country(country):
    if not isinstance(country, str):
        return "Not Found"

    country = country.split("/")[-1].strip()
    country = pycountry.countries.get(name=country)
    return "Not Found" if country is None else country.alpha_3


X["customer_country"] = [
    preprocess_country(country) for country in X["customer_country"]
]

X_test["customer_country"] = [
    preprocess_country(country) for country in X_test["customer_country"]
]

### Feature: Business Unit ("business_unit")

In [6]:
X["business_unit"] = X["business_unit"].astype(
    pd.CategoricalDtype(categories=["ID", "AS", "IT", "Solution", "CM"])
)
X_test["business_unit"] = X_test["business_unit"].astype(
    pd.CategoricalDtype(categories=["ID", "AS", "IT", "Solution", "CM"])
)

### Feature: Inquiry Type ("inquiry_type")

In [7]:
inquiry_type_remap = {
    # Quotation: Asking about the product itself.
    "Q": [
        "Quotation or Purchase Consultation",
        "Quotation or purchase consultation",
        "Quotation or Purchase consultation",
        "quotation_or_purchase_consultation",
        "Purchase or Quotation",
        "Purchase",
        "Request for quotation or purchase",
        "Sales Inquiry",
        "sales",
        "quotation_",
    ],
    # Usage: Asking about how to use the product.
    "U": [
        "Usage or Technical Consultation",
        "Usage or technical consultation",
        "usage or technical consultation",
        "usage_or_technical_consultation",
        "Technical Consultation",
        "Technical Support",
        "technical",
        "technical_consultation",
        "Request for technical consulting",
        "Product Information",
        "i want to know the details about it",
        "first Info and pricing",
        "Toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung",
        "tôi cần tham khảo giá và giải pháp từ LG",
        "Trainings",
    ],
    "O": [
        "Other",
        "Others",
        "other_",
        "other",
        "others",
        "Etc.",
        "ETC.",
        "Not specified",
        "(Select ID_Needs)",
        "IDB",
        "Services",
        "Request for Partnership",
        "Request a Demo",
        "Request for Distributorship",
        "Customer Suggestions",
        *(
            df_train["inquiry_type"]
            .value_counts()
            .loc[df_train["inquiry_type"].value_counts() < 10]
            .index.tolist()
        ),
    ],
}

for key, value in inquiry_type_remap.items():
    X.loc[X["inquiry_type"].isin(value), "inquiry_type"] = key
    X_test.loc[X_test["inquiry_type"].isin(value), "inquiry_type"] = key

### Feature: Customer Position ("customer_position")

In [8]:
X['customer_position'] = X['customer_position'].replace('decision maker', 'decision-maker')
X['customer_position'] = X['customer_position'].replace('decision influencer', 'decision-influencer')
X['customer_position'] = X['customer_position'].replace('architect/consultant', 'architecture/consult')
X['customer_position'] = X['customer_position'].replace('physics and mathematics teacher', 'math and physics teacher')
X['customer_position'] = X['customer_position'].replace('vicepresident', 'vice president')
X['customer_position'] = X['customer_position'].replace('others', 'other')

X_test['customer_position'] = X_test['customer_position'].replace('decision influencer', 'decision-influencer')
X_test['customer_position'] = X_test['customer_position'].replace('architect/consultant', 'architecture/consult')
X_test['customer_position'] = X_test['customer_position'].replace('physics and mathematics teacher', 'math and physics teacher')
X_test['customer_position'] = X_test['customer_position'].replace('vicepresident', 'vice president')
X_test['customer_position'] = X_test['customer_position'].replace('others', 'other')

In [9]:
# Classification by position and occupation
C_position_high_executive = [
    'manager', 'ceo/founder', 'c-level executive',
    'vice president', 'c-levelexecutive', 'founder',
    'commercial consultant', 'government',
    'operations', 'other - please specify - cedia association',
    'leadership/executive office/owner', 'market intelligence/research',
    'chairman', 'co-founder', 'chief executive officer',
    'ceo/fundador', 'gerente', 'president', 'genel müdür', 'bulgaria',
]

C_position_low_executive = [
    'director', 'intern', 'trainee', 'entry level',
    'employee', 'administrative', 'manufacturer',
    'system integrator', 'distributor', 'business unit director',
    'business development', 'product management', 'business partner',
    'subsidiary sales (ise)', 'business development/sales',
    'sales',
]

C_position_education = [
    'physics teacher', 'assistant professor',
    'maths lecturer', 'science teacher', 'physics faculty',
    'teacher/middle school coordinator', 'math and physics teacher',
    'professor of mathematics', 'prof.', 'professor of mathematics',
    'academic specialist', 'principal at oxford integrated pu science college'
    'assistant professor of English', 'educator', 'professor',
    'quantitative aptitude faculty', 'english trainer for ielts,toefl,pte,gre,sat exams.'
    'associate professor', 'principal & director', 'hon dean',
    'pgt physics', 'education professional', 'chemistry teacher',
    'director cum faculty at gaining apex coaching centre',
    'academic coordinator/ post graduate teacher (accountancy, business studies)/ tgt (ict)',
    'teacher', 'senior lecturer', 'neet/ olympiad expert faculty',
    'asst prof.', 'associate professor in electronics engg', 'professional trainer',
    'education',
]

C_position_hospital = [
    'hospital', 'medical device manufacturer', 'vp', 'medical imaging specialist',
    'tierarzt', 'főorvos', 'surgery professional', 'pathologist', 'radiology professional'
]

C_position_others = [
    'entry level', 'customer', 'technical',
    'exhibition', 'end-user', 'cargo', 'técnico', 'exhibitiontv',
    'proprietário(a)', 'mindenes', 'research', 'the big boss',
    'partner', 'associate/analyst', 'none', 'unpaid', 'av management',
    'engineering', 'installer', 'homeowner', 'commercial end-user'
    'lider de desarrollo', 'entrepreneurship', 'not applicable',
    'no influence',
]

C_position_consultant = [
    'consultant', 'software/solution provider', 'guest faculty',
    'career coach', 'consulting',
]


In [10]:
position_category= {
    'High Executive': C_position_high_executive,
    'Low Executive': C_position_low_executive,
    'Education': C_position_education,
    'Hospital': C_position_hospital,
    'Consultant': C_position_consultant,
    'Others': C_position_others
}

for category, positions in position_category.items():
    X.loc[X['customer_position'].isin(positions), 'C_position_1'] = category
    X_test.loc[X_test['customer_position'].isin(positions), 'C_position_1'] = category

# 'Hospital', 'Consultant', 'Education'는 무의미한 데이터로 판단 -> Others로 대체
categories_drop = ['Hospital', 'Consultant', 'Education']
X.loc[X['C_position_1'].isin(categories_drop), 'C_position_1'] = 'Others'
X_test.loc[X_test['C_position_1'].isin(categories_drop), 'C_position_1'] = 'Others'

# C_position_category one_hot_encoding
X_encoded = pd.get_dummies(X, columns=['C_position_1'])
X_test_encoded = pd.get_dummies(X_test, columns=['C_position_1'])

col =['C_position_1_High Executive', 'C_position_1_Low Executive', 'C_position_1_Others', 'C_position_1_nan']


X[col] = X_encoded[col].astype(int)
X_test[col] = X_test_encoded[col].astype(int)

# 기존 열 drop
X.drop(['C_position_1'], axis=1, inplace=True)
X.drop(['customer_position'], axis=1, inplace=True)

X_test.drop(['C_position_1'], axis=1, inplace=True)
X_test.drop(['customer_position'], axis=1, inplace=True)

# 결측치 처리
X.loc[X['C_position_1_nan'] == 1, 'C_position_1_High Executive'] = 1
X.drop(['C_position_1_nan'], axis=1, inplace=True)
X_test.loc[X_test['C_position_1_nan'] == 1, 'C_position_1_High Executive'] = 1
X_test.drop(['C_position_1_nan'], axis=1, inplace=True)

## 3. Feature Engineering

In [11]:
country_frequency = X["customer_country"].value_counts(normalize=True)
X["customer_country_frequency"] = X["customer_country"].map(country_frequency)
X_test["customer_country_frequency"] = X_test["customer_country"].map(country_frequency)

country_target = (
    pd.concat([X, y], axis=1).groupby("customer_country")["is_converted"].mean()
)
X["customer_country_target"] = X["customer_country"].map(country_target)
X_test["customer_country_target"] = X_test["customer_country"].map(country_target)

owner_frequency = X["lead_owner"].value_counts(normalize=True)
X["lead_owner_frequency"] = X["lead_owner"].map(owner_frequency)
X_test["lead_owner_frequency"] = X_test["lead_owner"].map(owner_frequency)

owner_target = pd.concat([X, y], axis=1).groupby("lead_owner")["is_converted"].mean()
X["lead_owner_target"] = X["lead_owner"].map(owner_target)
X_test["lead_owner_target"] = X_test["lead_owner"].map(owner_target)

## 4. Feature Encoding

In [12]:
def encode_features(
    df: pd.DataFrame,
    features=["enterprise", "business_unit", "inquiry_type", "bant_submit"],
) -> pd.DataFrame:
    df_encoded = pd.get_dummies(df[features], columns=features)
    df_encoded = df_encoded.apply(lambda x: x.astype("category").cat.codes)
    df = pd.concat([df, df_encoded], axis=1).drop(features, axis=1)
    return df


X = encode_features(X)
X_test = encode_features(X_test)

## 5. Feature Normalization

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

standard_transformer = Pipeline(steps=[("standardize", StandardScaler())])
column_transformer = ColumnTransformer(
    transformers=[
        ("std", standard_transformer, ["lead_desc_length", "historical_existing_cnt"]),
    ],
    verbose_feature_names_out=False,
    remainder="passthrough",
)
column_transformer.set_output(transform="pandas")

X = column_transformer.fit_transform(X)
X_test = column_transformer.transform(X_test)

## 6. Outlier Removal

## 7. Feature Removal

In [14]:
features_to_drop = [
    "customer_country",
    "business_subarea",
    "business_area",
    "customer_idx",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_job",
    "customer_type",
    "response_corporate",
    "expected_timeline",
    "lead_owner",
]

X = X.drop(features_to_drop, axis=1)  # type: ignore
X_test = X_test.drop(features_to_drop, axis=1)  # type: ignore

# Modeling

In [15]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(random_state=RANDOM_STATE, verbose=False)

# K-Fold Cross Validation

In [16]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score, roc_auc_score)
from sklearn.model_selection import StratifiedKFold

kfolds, scores = (
    StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True),
    [],
)

for fold_idx, (train_idx, val_idx) in enumerate(kfolds.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    sm = SMOTE(sampling_strategy="minority", random_state=RANDOM_STATE)
    X_train, y_train = sm.fit_resample(X_train, y_train)  # type: ignore

    model.fit(X_train, y_train)
    prediction = model.predict(X_val)
    # print(prediction)
    # exit()
    prediction = np.where(prediction == "True", True, False)

    print(f"Accuracy (Fold {fold_idx}): {accuracy_score(y_val, prediction)}")
    print(f"Precision (Fold {fold_idx}): {precision_score(y_val, prediction)}")
    print(f"Recall (Fold {fold_idx}): {recall_score(y_val, prediction)}")
    print(f"F1-Score (Fold {fold_idx}): {f1_score(y_val, prediction)}")
    print(
        f"ROC-AUC Score (Fold {fold_idx}): "
        + f"{roc_auc_score(y_val, model.predict_proba(X_val)[:,1])}\n"
    )

    scores.append(f1_score(y_val, prediction))

print(f"Cross-Validation Average F1-Score: {np.mean(scores)}")

Accuracy (Fold 0): 0.9397133220910624
Precision (Fold 0): 0.6058091286307054
Recall (Fold 0): 0.7525773195876289
F1-Score (Fold 0): 0.6712643678160918
ROC-AUC Score (Fold 0): 0.9661958384217053

Accuracy (Fold 1): 0.9430016863406409
Precision (Fold 1): 0.6210873146622735
Recall (Fold 1): 0.777319587628866
F1-Score (Fold 1): 0.6904761904761905
ROC-AUC Score (Fold 1): 0.9716069788797062

Accuracy (Fold 2): 0.9411467116357505
Precision (Fold 2): 0.6096774193548387
Recall (Fold 2): 0.7793814432989691
F1-Score (Fold 2): 0.6841628959276017
ROC-AUC Score (Fold 2): 0.9669134645423305

Accuracy (Fold 3): 0.9412310286677908
Precision (Fold 3): 0.6101694915254238
Recall (Fold 3): 0.7793814432989691
F1-Score (Fold 3): 0.6844726120416479
ROC-AUC Score (Fold 3): 0.9680258063294613

Accuracy (Fold 4): 0.9376001349186271
Precision (Fold 4): 0.5942622950819673
Recall (Fold 4): 0.7474226804123711
F1-Score (Fold 4): 0.6621004566210045
ROC-AUC Score (Fold 4): 0.9635724787996588

Cross-Validation Average F

# Create Submission

In [17]:
sm = SMOTE(sampling_strategy="minority", random_state=RANDOM_STATE)
X, y = sm.fit_resample(X, y)  # type: ignore
model.fit(X, y)

prediction = model.predict(X_test)
prediction = np.where(prediction == "True", True, False)

df_submission["is_converted"] = prediction
df_submission.to_csv("submission.csv", index=False)
print(df_submission["is_converted"].value_counts())
df_submission.head()

is_converted
False    4321
True      950
Name: count, dtype: int64


,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
